# SQL Server 2019 Demo 2 - Data Virtualization - Using Polybase to query Azure SQL Database
This notebook contains an example of how to use external tables to query data in Azure SQL Database **without moving data**. 

This notebook assumes you are using SQL Server 2019 and that the Polybase feature has been installed and enabled.

## Step 1: Restore the Wide Importers datbase

This notebook uses the sample WideWorldImporters sample database in SQL 2019.  The database can be downloaded at the GitHub repository [SQL Server Demos - World Wide Importers](https://github.com/Microsoft/sql-server-samples/releases/tag/wide-world-importers-v1.0).

## Step 2: Create a database in Azure SQL, table, and add data

Create a database in Azure SQL called **AzureDemoDB**. Execute the following T-SQL to create a table and insert data into the database.


In [1]:
DROP TABLE IF EXISTS [ModernStockItems]
GO
CREATE TABLE [ModernStockItems](
	[StockItemID] [int] NOT NULL,
	[StockItemName] [nvarchar](100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[SupplierID] [int] NOT NULL,
	[ColorID] [int] NULL,
	[UnitPackageID] [int] NOT NULL,
	[OuterPackageID] [int] NOT NULL,
	[Brand] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[Size] [nvarchar](20) COLLATE Latin1_General_100_CI_AS NULL,
	[LeadTimeDays] [int] NOT NULL,
	[QuantityPerOuter] [int] NOT NULL,
	[IsChillerStock] [bit] NOT NULL,
	[Barcode] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[UnitPrice] [decimal](18, 2) NOT NULL,
	[RecommendedRetailPrice] [decimal](18, 2) NULL,
	[TypicalWeightPerUnit] [decimal](18, 3) NOT NULL,
	[LastEditedBy] [int] NOT NULL,
CONSTRAINT [PK_Warehouse_StockItems] PRIMARY KEY CLUSTERED 
(
	[StockItemID] ASC
)
)
GO
-- Now insert some data. We don't coordinate with unique keys in WWI on SQL Server
-- so pick numbers way larger than exist in the current StockItems in WWI which is only 227
INSERT INTO ModernStockItems VALUES
(100000,
'Dallas Cowboys Jersey',
5,
4, -- Blue
4, -- Box
4, -- Bob
'Under Armour',
'L',
30,
1,
0,
'123456789',
2.0,
50,
75,
2.0,
1
)
GO

Commands completed successfully.

Commands completed successfully.

(1 row affected)

Total execution time: 00:00:00.384

# Step 3 Create a user to access the Azure SQL DB

For ease of demo use, we will use a SQL Login.   The following commands need to be run against the master and the AzureSQLDB in order to create a new user.

For ease of use, Run this script SQL Server Management Studio as part of the setup.


In [0]:

--Open a new query window connected to Master database (right-click on Master and click New Query) 
--and execute the following command:

Use [Master]
GO

Create Login usgsloader with PASSWORD = '<Password!1234>'
GO

-- Open a new query window connected to Master database (right-click on Master and click New Query) 
-- and execute the following command:


Use [AzureSQLDB]
go
Create user usgsloader from login usgsloader
go
EXEC sp_addrolemember 'db_owner', 'usgsloader';
go

## Step 4A: Create a master key in your SQL 2019 Instance

Change your connection to your SQL 2019 Instance.

Create a master key to encrypt the database credential

In [4]:
-- Use [WideWorldImporters]

CREATE MASTER KEY ENCRYPTION BY PASSWORD = '@Password1234!!'--'<password>'
GO

: Msg 15578, Level 16, State 1, Line 3
There is already a master key in the database. Please drop it before performing this statement.

Total execution time: 00:00:00.067

## Step 4B: Validate that Polybase is enabled.

See the Referenced for the steps to enable Polybase in your SQL 2019 instance.  [SQL 2019 Polybase Enabled](https://docs.microsoft.com/en-us/sql/relational-databases/polybase/polybase-installation?view=sql-server-ver15)

Run the following Script to validate that these are enabled.

If using Azure Data Studio, you can change the Attach TO, in order to change the 


In [6]:
exec sp_configure @configname = 'polybase enabled', @configvalue = 1;
RECONFIGURE;

--Confirm Run the following command. If PolyBase is installed, the return is 1. Otherwise, it's 0.
SELECT SERVERPROPERTY ('IsPolyBaseInstalled') AS IsPolyBaseInstalled;

Configuration option 'polybase enabled' changed from 1 to 1. Run the RECONFIGURE statement to install.

(1 row affected)

Total execution time: 00:00:00.050

IsPolyBaseInstalled
1


## Step 5: Create a database credential.
The database credential contains the IDENTITY (login) and SECRET (password) of the remote Azure SQL Database server or Managed Instance. Change this to the login and password for your server. [Configure PolyBase to access external data in SQL Server](https://docs.microsoft.com/en-us/sql/relational-databases/polybase/polybase-configure-sql-server?view=sql-server-ver15)

This should already be completed by default, but run this command in the WideWorldImporters first.

```
USE [WideWorldImporters]
GO
CREATE MASTER KEY ENCRYPTION BY PASSWORD = 'Thisisa@password'
GO
```


Format
```
CREATE DATABASE SCOPED CREDENTIAL AzureSQLDatabaseCredentials   
WITH IDENTITY = '<login>', SECRET = '<password>'
```

In [19]:
--Drop DATABASE SCOPED CREDENTIAL AzureSQLDatabaseCredentials2   


CREATE DATABASE SCOPED CREDENTIAL AzureSQLDatabaseCredentials2   
WITH IDENTITY = '<<user>>', SECRET = '<<ComplexPassword!1234>>'
GO

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:00.016

## Step 6: Create an EXTERNAL DATA SOURCE
The EXTERNAL DATA SOURCE indicates what type of data source, the connection "string", where PUSHDOWN predicates should be used (if possible), and the name of the database credential.

The LOCATION syntax is ``` datasourcetype:connection string ```

datasourcetype can be sqlserver, oracle, teradata, mongodb, or odbc (Windows only)
The connection string depends on the datasourcetype

For this example, put in the name of the Azure SQL Server database server or Managed instance

```
CREATE EXTERNAL DATA SOURCE AzureSQLDatabase
WITH ( 
LOCATION = 'sqlserver://<server name>.database.windows.net',
PUSHDOWN = ON,
CREDENTIAL = AzureSQLDatabaseCredentials
)
GO
```

In [1]:
CREATE EXTERNAL DATA SOURCE AzureSQLDatabaseDemo
WITH ( 
LOCATION = 'sqlserver://seylabdb.database.windows.net',
PUSHDOWN = ON,
CREDENTIAL = AzureSQLDatabaseCredentials2
)
GO

: Msg 46721, Level 20, State 1, Line 1
Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication.

Total execution time: 00:00:00.050

## Step 7: Create a schema for the EXTERNAL TABLE
Schemas provide convenient methods to secure and organize objects

In [5]:
CREATE SCHEMA azuresqldb
GO

Commands completed successfully.

Total execution time: 00:00:00.004

## Step 8: Create an EXTERNAL TABLE
An external table provides metadata so SQL Server knows how to map columns to the remote table. The name of the table for the external table can be your choice. But the columns must be specified in the same order with the same name as they are defined in the remote table. Furthermore, local data types must be compatible with the remote table.

** Note that there was an error running this from the Azure Data Studio Notebook, works from the SQL Script **


The WITH clause specifies a LOCATION. This LOCATION is different than the EXTERNAL DATA SOURCE. This LOCATION indicates the [database].[schema].[table] of the remote table. The DATA_SOURCE clauses is the name of the EXTERNAL DATA SOURCE you created earlier.

In [4]:
DROP EXTERNAL TABLE azuresqldb.ModernStockItems
go 
CREATE EXTERNAL TABLE azuresqldb.ModernStockItems
(
	[StockItemID] [int] NOT NULL,
	[StockItemName] [nvarchar](100) COLLATE Latin1_General_100_CI_AS NOT NULL,
	[SupplierID] [int] NOT NULL,
	[ColorID] [int] NULL,
	[UnitPackageID] [int] NOT NULL,
	[OuterPackageID] [int] NOT NULL,
	[Brand] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[Size] [nvarchar](20) COLLATE Latin1_General_100_CI_AS NULL,
	[LeadTimeDays] [int] NOT NULL,
	[QuantityPerOuter] [int] NOT NULL,
	[IsChillerStock] [bit] NOT NULL,
	[Barcode] [nvarchar](50) COLLATE Latin1_General_100_CI_AS NULL,
	[TaxRate] [decimal](18, 3) NOT NULL,
	[UnitPrice] [decimal](18, 2) NOT NULL,
	[RecommendedRetailPrice] [decimal](18, 2) NULL,
	[TypicalWeightPerUnit] [decimal](18, 3) NOT NULL,
	[LastEditedBy] [int] NOT NULL
)
 WITH (
 LOCATION='AzureDemoDB.dbo.ModernStockItems',
 DATA_SOURCE=AzureSQLDatabaseDemo
)
GO

: Msg 15151, Level 16, State 1, Line 1
Cannot drop the EXTERNAL TABLE 'azuresqldb.ModernStockItems', because it does not exist or you do not have permission.

: Msg 46721, Level 20, State 1, Line 3
Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication.

Total execution time: 00:00:00.009

## Step 9: Create statistics
SQL Server allows you to store local statistics about specific columns from the remote table. This can help the query processing to make more efficient plan decisions.

In [3]:
CREATE STATISTICS ModernStockItemsStats ON azuresqldb.ModernStockItems ([StockItemID]) WITH FULLSCAN
GO

: Msg 46721, Level 20, State 1, Line 1
Login failed. The login is from an untrusted domain and cannot be used with Integrated authentication.

Total execution time: 00:00:00.053

## Step 10: Try to scan the remote table
Run a simple query on the EXTERNAL TABLE to scan all rows.

In [1]:
SELECT * FROM azuresqldb.ModernStockItems
GO

: Msg 208, Level 16, State 1, Line 1
Invalid object name 'azuresqldb.ModernStockItems'.

Total execution time: 00:00:00.001

## Step 11: Query the remote table with a WHERE clause
Even though the table may be small SQL Server will "push" the WHERE clause filter to the remote table

In [9]:
SELECT * FROM azuresqldb.ModernStockItems WHERE StockItemID = 100000
GO

(1 row affected)

Total execution time: 00:00:01.612

StockItemID,StockItemName,SupplierID,ColorID,UnitPackageID,OuterPackageID,Brand,Size,LeadTimeDays,QuantityPerOuter,IsChillerStock,Barcode,TaxRate,UnitPrice,RecommendedRetailPrice,TypicalWeightPerUnit,LastEditedBy
100000,Dallas Cowboys Jersey,5,4,4,4,Under Armour,L,30,1,0,123456789,2.000,50.00,75.00,2.000,1


## Step 12: Join with local SQL Server tables
Use a UNION to find all stockitems for a specific supplier both locally and in the Azure table

In [10]:
SELECT msi.StockItemName, msi.Brand, c.ColorName
FROM azuresqldb.ModernStockItems msi
JOIN [Purchasing].[Suppliers] s
ON msi.SupplierID = s.SupplierID
and s.SupplierName = 'Graphic Design Institute'
JOIN [Warehouse].[Colors] c
ON msi.ColorID = c.ColorID
UNION
SELECT si.StockItemName, si.Brand, c.ColorName
FROM [Warehouse].[StockItems] si
JOIN [Purchasing].[Suppliers] s
ON si.SupplierID = s.SupplierID
and s.SupplierName = 'Graphic Design Institute'
JOIN [Warehouse].[Colors] c
ON si.ColorID = c.ColorID
GO

(43 rows affected)

Total execution time: 00:00:00.306

StockItemName,Brand,ColorName
Dallas Cowboys Jersey,Under Armour,Blue
DBA joke mug - daaaaaa-ta (Black),NULL,Black
DBA joke mug - daaaaaa-ta (White),NULL,White
DBA joke mug - I will get you in order (Black),NULL,Black
DBA joke mug - I will get you in order (White),NULL,White
DBA joke mug - it depends (Black),NULL,Black
DBA joke mug - it depends (White),NULL,White
DBA joke mug - mind if I join you? (Black),NULL,Black
DBA joke mug - mind if I join you? (White),NULL,White
DBA joke mug - SELECT caffeine FROM mug (Black),NULL,Black


# References (Need to update)


[Configure PolyBase to access external data in Azure Blob Storage](https://docs.microsoft.com/en-us/sql/relational-databases/polybase/polybase-configure-azure-blob-storage?view=sql-server-ver15)

[SQL 2019 Polybase Enable](https://docs.microsoft.com/en-us/sql/relational-databases/polybase/polybase-installation?view=sql-server-ver15)